## Setup
This notebook expects to be ran in the same directory as safegraphs uncompressed freely available safegraph_open_census_data folder (https://www.safegraph.com/open-census-data) and the csv of Philadelphia crime incidents (http://metadata.phila.gov/#home/datasetdetails/5543868920583086178c4f8e/representationdetails/570e7621c03327dc14f4b68d/). Additional data has been purchased from Safegraph's data shop on (https://shop.safegraph.com/) consisting of all locations they have recorded for October 2019 in Philadelphia that fall under their category 'Limited Service Restaurants' and this uncompressed data folder also sits next to the notebook. Due to size constraints in the first two cases, and proprietary nature in the second case, both these data sets must be obtained on the part of the one exercising this notebook, though the final output csv has been included in this folder.

In [163]:
import ijson
import json
from json.decoder import JSONDecodeError
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import requests
import urllib.request

## Census Wrangling
For our investigation we want to use just Census Block Groups within Philadelphia. Census observations are available in the census data folder with a seperate table for each observation type. For a subset of tables we choose as our predictors, we will filter to just Philadelphia census blocks, then create summary Series for the table's observation of interest. We will then combine the disparate series into one data frame.

In [6]:
# First we use the census block geometry data to determine which census
# block groups are within Philadelphia, Due to file size we must stream
# this json file to pull the CBG codes in question rather than load as
# a whole
with open('safegraph_open_census_data/geometry/cbg.geojson', 'r') as geosfd:
    cbgs = ijson.items(geosfd, 'features.item')
    philadelphia_cbgs = [
      cbg['properties']['CensusBlockGroup'] for cbg in cbgs
        if cbg['properties']['County'] == 'Philadelphia County' and
           cbg['properties']['State'] == 'PA']
    
print(len(philadelphia_cbgs))

1336


### Gathering census variables for Philadelphia CBGs
We will be using metadata/cbg_field_descriptions.csv to discover the releveant columns in our census files for the variables we would like to inspect. We capture each variable from the census data files as follows.

In [136]:
# Variable: Total Population
total_pop_df = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b01.csv', index_col=0, usecols=[0, 1])
total_pop_df.columns = ['Total Population']
total_pop_df = total_pop_df[total_pop_df.index.isin(philadelphia_cbgs)]
total_pop_df.head()

,Total Population
census_block_group,
421010001001,1762
421010001002,2250
421010002001,2563
421010003001,942
421010003002,2588


In [137]:
# Variable: Worked from Home
# File Column Number - Census Identifier Code - Description
# 523 - B08301e21 - Worked at home: Workers 16 years and over -- 
#                   (Estimate)
pop_wfh = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b08.csv',
    index_col=0, usecols=[0,523])
pop_wfh.columns = ['Pop. that Works from Home']
pop_wfh = pop_wfh[pop_wfh.index.isin(philadelphia_cbgs)]

In [138]:
pop_wfh.head()

,Pop. that Works from Home
census_block_group,
421010001001,47
421010001002,59
421010002001,8
421010003001,45
421010003002,102


In [153]:
# Variable: Single Occupant Households
# File Column Number - Census Identifier Code - Description
# 15 - B11001e8 - HOUSEHOLD TYPE (INCLUDING LIVING ALONE): Nonfamily
#                 households: Householder living alone: Households --
#                 (Estimate)
living_alone = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b11.csv', index_col=0, usecols=[0,15])
living_alone.columns = ['Single Occupant Households']
living_alone = living_alone[living_alone.index.isin(philadelphia_cbgs)]
living_alone.head()

,Single Occupant Households
census_block_group,
421010001001,634
421010001002,564
421010002001,313
421010003001,219
421010003002,616


In [140]:
# Variable: Single Parent Households (at least one minor present)
# We must sum male and female single parent households for this
# File Column Number - Census Identifier Code - Description
# 479 - B11004e10 - FAMILY TYPE BY PRESENCE AND AGE OF RELATED CHILDREN
#                   UNDER 18 YEARS: Other family: Male householder, no
#                   wife present: With related children of the
#                   householder under 18 years: Families -- (Estimate)
# 491 - B11004e16 - FAMILY TYPE BY PRESENCE AND AGE OF RELATED CHILDREN
#                   UNDER 18 YEARS: Other family: Female householder,
#                   no husband present: With related children of the
#                   householder under 18 years: Families -- (Estimate)
single_parent = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b11.csv',
    index_col=0, usecols=[0,479,491])
single_parent = single_parent[single_parent.index.isin(philadelphia_cbgs)]
single_parent['Single Parent Households'] =\
    single_parent['B11004e10'] + single_parent['B11004e16']
single_parent = single_parent.drop(['B11004e10','B11004e16'], axis=1)
single_parent.head()

,Single Parent Households
census_block_group,
421010001001,13
421010001002,0
421010002001,0
421010003001,0
421010003002,26


In [141]:
# Variable: Both Parent Households (at least one minor present)
# File Column Number - Census Identifier Code - Description
# 15 - B11004e3 - FAMILY TYPE BY PRESENCE AND AGE OF RELATED CHILDREN
#                 UNDER 18 YEARS: Married-couple family: With related
#                 children of the householder under 18 years: Families
#                 -- (Estimate)
married_parents = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b11.csv',
    index_col=0, usecols=[0,465])
married_parents.columns = ['Married Parent Households']
married_parents =\
    married_parents[married_parents.index.isin(philadelphia_cbgs)]
married_parents.head()

,Married Parent Households
census_block_group,
421010001001,45
421010001002,51
421010002001,195
421010003001,18
421010003002,117


In [142]:
# Variable: School Enrollment Undergrad and Grad
# Sum different genders and degrees pursued
# File Column Number - Census Identifier Code - Description
# 37 - B14002e19 - SEX BY SCHOOL ENROLLMENT BY LEVEL OF SCHOOL BY TYPE
#                  OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER: Male:
#                  Enrolled in school: Enrolled in college
#                  undergraduate years: Population 3 years and over --
#                  (Estimate)
# 43 - B14002e22 - SEX BY SCHOOL ENROLLMENT BY LEVEL OF SCHOOL BY TYPE
#                  OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER: Male:
#                  Enrolled in school: Enrolled in graduate or
#                  professional school: Population 3 years and over --
#                  (Estimate)
# 85 - B14002e43 - SEX BY SCHOOL ENROLLMENT BY LEVEL OF SCHOOL BY TYPE
#                  OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER:
#                  Female: Enrolled in school: Enrolled in college
#                  undergraduate years: Population 3 years and over --
#                  (Estimate)
# 91 - B14002e46 - SEX BY SCHOOL ENROLLMENT BY LEVEL OF SCHOOL BY TYPE
#                  OF SCHOOL FOR THE POPULATION 3 YEARS AND OVER:
#                  Female: Enrolled in school: Enrolled in graduate or
#                  professional school: Population 3 years and over --
#                  (Estimate)
school_enrollment = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b14.csv',
    index_col=0, usecols=[0,37,43,85,91])
school_enrollment =\
    school_enrollment[school_enrollment.index.isin(philadelphia_cbgs)]
school_enrollment['Undergrads and Grads'] = school_enrollment.sum(axis=1)
school_enrollment = school_enrollment.drop(
    ['B14002e19', 'B14002e22', 'B14002e43', 'B14002e46'], axis=1)
school_enrollment.head()

,Undergrads and Grads
census_block_group,
421010001001,236
421010001002,202
421010002001,445
421010003001,260
421010003002,294


In [143]:
# Variable: High School / GED Educational Attainment
# Sum diploma and GED counts
# File Column Number - Census Identifier Code - Description
# 103 - B15003e17 - EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS
#                   AND OVER: Regular high school diploma: Population
#                   25 years and over -- (Estimate)
# 105 - B15003e18 - EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS
#                   AND OVER: GED or alternative credential: Population
#                   25 years and over -- (Estimate)
high_school_achieved = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b15.csv',
    index_col=0, usecols=[0,103,105])
high_school_achieved =\
    high_school_achieved[high_school_achieved.index.isin(philadelphia_cbgs)]
high_school_achieved['Diploma/GED attained'] = high_school_achieved.sum(axis=1)
high_school_achieved =\
    high_school_achieved.drop(['B15003e17', 'B15003e18'], axis=1)
high_school_achieved.head()

,Diploma/GED attained
census_block_group,
421010001001,74
421010001002,167
421010002001,308
421010003001,21
421010003002,259


In [144]:
# Variable: Attained Degree
# Sum Bachelors, Masters, and Doctorate degree counts
# File Column Number - Census Identifier Code - Description
# 113 - B15003e22 - EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS
#                   AND OVER: Bachelor's degree: Population 25 years
#                   and over -- (Estimate)
# 115 - B15003e23 - EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS
#                   AND OVER: Master's degree: Population 25 years and
#                   over -- (Estimate)
# 119 - B15003e25 - EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS
#                   AND OVER: Doctorate degree: Population 25 years and
#                   over -- (Estimate)
degree_achieved = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b15.csv',
    index_col=0, usecols=[0,113,115,119])
degree_achieved =\
    degree_achieved[degree_achieved.index.isin(philadelphia_cbgs)]
degree_achieved['Degree attained'] = degree_achieved.sum(axis=1)
degree_achieved =\
    degree_achieved.drop(['B15003e22', 'B15003e23', 'B15003e25'], axis=1)
degree_achieved.head()

,Degree attained
census_block_group,
421010001001,1145
421010001002,1405
421010002001,955
421010003001,442
421010003002,1331


In [145]:
# Variable: Households in Poverty
# File Column Number - Census Identifier Code - Description
# 109 - B17017e1 - POVERTY STATUS IN THE PAST 12 MONTHS BY HOUSEHOLD
#                  TYPE BY AGE OF HOUSEHOLDER: Total: Households --
#                  (Estimate)
houses_in_poverty = pd.read_csv(
    'safegraph_open_census_data/data/cbg_c17.csv',
    index_col=0, usecols=[0,109])
houses_in_poverty.columns = ['Households in Poverty']
houses_in_poverty =\
    houses_in_poverty[houses_in_poverty.index.isin(philadelphia_cbgs)]
houses_in_poverty.head()

,Households in Poverty
census_block_group,
421010001001,1123
421010001002,1182
421010002001,950
421010003001,437
421010003002,1328


In [146]:
# Variable: Income 40k and below
# Sum all income counts below 40k
# File Column Number - Census Identifier Code - Description
# 3 - B19001e2 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                INFLATION-ADJUSTED DOLLARS): Less than $10,000:
#                Households -- (Estimate)
# 5 - B19001e3 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                INFLATION-ADJUSTED DOLLARS): $10,000 to $14,999:
#                Households -- (Estimate)
# 7 - B19001e4 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                INFLATION-ADJUSTED DOLLARS): $15,000 to $19,999:
#                Households -- (Estimate)
# 9 - B19001e5 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                INFLATION-ADJUSTED DOLLARS): $20,000 to $24,999:
#                Households -- (Estimate)
# 11 - B19001e6 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                 INFLATION-ADJUSTED DOLLARS): $25,000 to $29,999:
#                 Households -- (Estimate)
# 13 - B19001e7 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                 INFLATION-ADJUSTED DOLLARS): $30,000 to $34,999:
#                 Households -- (Estimate)
# 15 - B19001e8 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                 INFLATION-ADJUSTED DOLLARS): $35,000 to $39,999:
#                 Households -- (Estimate)
below_40k = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b19.csv',
    index_col=0, usecols=[0,3,5,7,9,11,13,15])
below_40k = below_40k[below_40k.index.isin(philadelphia_cbgs)]
below_40k['Below 40k Earners'] = below_40k.sum(axis=1)
below_40k = below_40k.drop(
    ['B19001e2', 'B19001e3', 'B19001e4', 'B19001e5',
     'B19001e6', 'B19001e7', 'B19001e8'],
    axis=1)
below_40k.head()

,Below 40k Earners
census_block_group,
421010001001,257
421010001002,174
421010002001,425
421010003001,117
421010003002,373


In [147]:
# Vareiable: Income 40k to 100k
# Sum all incomes in range
# File Column Number - Census Identifier Code - Description
# 17 - B19001e9 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                 INFLATION-ADJUSTED DOLLARS): $40,000 to $44,999:
#                 Households -- (Estimate)
# 19 - B19001e10 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                  INFLATION-ADJUSTED DOLLARS): $45,000 to $49,999:
#                  Households -- (Estimate)
# 21 - B19001e11 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                  INFLATION-ADJUSTED DOLLARS): $50,000 to $59,999:
#                  Households -- (Estimate)
# 23 - B19001e12 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                  INFLATION-ADJUSTED DOLLARS): $60,000 to $74,999:
#                  Households -- (Estimate)
# 25 - B19001e13 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                  INFLATION-ADJUSTED DOLLARS): $75,000 to $99,999:
#                  Households -- (Estimate)
between_40_100k = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b19.csv',
    index_col=0, usecols=[0, 17, 19, 21, 23, 25])
between_40_100k =\
    between_40_100k[between_40_100k.index.isin(philadelphia_cbgs)]
between_40_100k['40k-100k Earners'] = between_40_100k.sum(axis=1)
between_40_100k = between_40_100k.drop(
    ['B19001e9', 'B19001e10', 'B19001e11', 'B19001e12', 'B19001e13'], axis=1)
between_40_100k.head()

,40k-100k Earners
census_block_group,
421010001001,279
421010001002,397
421010002001,225
421010003001,82
421010003002,359


In [148]:
# Variable: Above 100k Earners
# Sum above 100k earner variables
# File Column Number - Census Identifier Code - Description
# 27 - B19001e14 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                  INFLATION-ADJUSTED DOLLARS): $100,000 to $124,999:
#                  Households -- (Estimate)
# 29 - B19001e15 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                  INFLATION-ADJUSTED DOLLARS): $125,000 to $149,999:
#                  Households -- (Estimate)
# 31 - B19001e16 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                  INFLATION-ADJUSTED DOLLARS): $150,000 to $199,999:
#                  Households -- (Estimate)
# 33 - B19001e17 - HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016
#                  INFLATION-ADJUSTED DOLLARS): $200,000 or more:
#                  Households -- (Estimate)
above_100k = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b19.csv',
    index_col=0, usecols=[0, 27, 29, 31, 33])
above_100k = above_100k[above_100k.index.isin(philadelphia_cbgs)]
above_100k['Above 100k Earners'] = above_100k.sum(axis=1)
above_100k = above_100k.drop(
    ['B19001e14', 'B19001e15', 'B19001e16', 'B19001e17'], axis=1)
above_100k.head()

,Above 100k Earners
census_block_group,
421010001001,587
421010001002,611
421010002001,300
421010003001,238
421010003002,596


In [149]:
# Variable: Unemployed Count
# File Column Number - Census Identifier Code - Description
# 487 - B23025e5 - EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND
#                  OVER: In labor force: Civilian labor force:
#                  Unemployed: Population 16 years and over --
#                  (Estimate)
unemployed = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b23.csv',
    index_col=0, usecols=[0,487])
unemployed.columns = ['Unemployed']
unemployed = unemployed[unemployed.index.isin(philadelphia_cbgs)]
unemployed.head()

,Unemployed
census_block_group,
421010001001,0
421010001002,82
421010002001,119
421010003001,30
421010003002,72


In [150]:
# Variable: Renter Occupied Households
# File Column Number - Census Identifier Code - Description
# 13 - B25003e3 - TENURE: Renter occupied: Occupied housing units --
#                 (Estimate)
renter_occupied = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b25.csv', index_col=0, usecols=[0,13])
renter_occupied.columns = ['Renter Occupied Households']
renter_occupied =\
    renter_occupied[renter_occupied.index.isin(philadelphia_cbgs)]
renter_occupied.head()

,Renter Occupied Households
census_block_group,
421010001001,853
421010001002,708
421010002001,592
421010003001,428
421010003002,898


In [151]:
# Variable: Owner Occupied Households
# 11 - B25003e2 - TENURE: Owner occupied: Occupied housing units --
#                 (Estimate)
owner_occupied = pd.read_csv(
    'safegraph_open_census_data/data/cbg_b25.csv',
    index_col=0, usecols=[0,11])
owner_occupied.columns = ['Owner Occupied Households']
owner_occupied = owner_occupied[owner_occupied.index.isin(philadelphia_cbgs)]
owner_occupied.head()

,Owner Occupied Households
census_block_group,
421010001001,270
421010001002,474
421010002001,358
421010003001,9
421010003002,430


## Additional Location Data
We can obtain some additional information beyond that available in the census about our Philadelphia geographies that could be helpful in determining the attractiveness of a location for fast food customers. In particular, we will add a count of crime incidences for each CBG, as well as measures of transit accessibility and walkability from the center of each CBG as provided by Walk Score (https://www.walkscore.com/).

### Crime Data
Beyond census data of a location, the occurence of crime in CBGs is likely to be a factor in the amount of traffic to a business in that area, with areas of lower incidence of crime being expected to attract more business, as well as incur less risk on the part of a business owner. Philadelphia provides crime incident report data at http://metadata.phila.gov/#home/datasetdetails/5543868920583086178c4f8e/representationdetails/570e7621c03327dc14f4b68d/ that we can line up with CBGs using the US Census Geocoding service on the provided lat/long of incidents. We will get a relative idea of the crime incidence in an area by totaling counts of all occurences in each CBG. While our dataset includes the year 2006 onwards, we will use the total count of incidents from just 2018 to get the most recent idea of the state of each area.

In [47]:
# First filter out just 2018 crimes
crime_occurences_ch = pd.read_csv(
    'data/incidents_part1_part2.csv',
    usecols=[6, 12, 13, 14],
    chunksize=1000)

chunk_list = []
for chunk in crime_occurences_ch:
    chunk_2018 = chunk[chunk.dispatch_date.str.startswith('2018')]
    
    if len(chunk_2018) > 0:
        chunk_list.append(chunk_2018)

crimes_2018 = pd.concat(chunk_list)

In [89]:
from json.decoder import JSONDecodeError

GEOCODE_COORD_API = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates'
query = {
    'benchmark':'Public_AR_Census2010',
    'vintage':'Census2010_Census2010',
    'layers':'14',
    'format':'json'
}

def get_cbg(record):
    query['x'] = record.point_x
    query['y'] = record.point_y
    try:
        resp = requests.get(GEOCODE_API, params = query).json()
        return resp['result']['geographies']['Census Blocks'][0]['GEOID'][:-3]
    except (IndexError, KeyError, JSONDecodeError) as e:
        return 0 # these missing cbgs will be tossed

In [56]:
crimes_2018 = crimes_2018.reset_index()
del(crimes_2018['index'])

In [ ]:
# We'll be requesting CBGs one at a time from our crime table's coordinates
crime_geocodes = []

In [74]:
# We'll be making one request at a time so we can resume making requests
# without lost data if there are connection errors to the geocoding service
for i, row, in crimes_2018.iterrows():
    crime_geocodes.append(get_cbg(row))

In [82]:
crimes_2018['CBG'] = crime_geocodes

In [114]:
cbg_crime_counts = crimes_2018['CBG'].value_counts()
cbg_crime_counts.index = cbg_crime_counts.index.astype(int)
cbg_crime_counts = cbg_crime_counts.reindex(philadelphia_cbgs)

cbg_crime_counts = cbg_crime_counts.to_frame()
cbg_crime_counts.columns = ['Crime Counts']

In [131]:
cbg_crime_counts.head()

,Crime Counts
421010001001,334
421010001002,178
421010002001,547
421010003001,587
421010003002,102


### Walk/Transit Score
And we can also get some information about the accessibility of each CBG from Walk Score's API (https://www.walkscore.com/) that could help us determine potential foot traffic and therefore business drawn.

In [ ]:
# get center coords for each CBG to start
cbg_scores = pd.read_csv(
    'safegraph_open_census_data/metadata/cbg_geographic_data.csv',
    index_col=0, usecols=[0,3,4])

cbg_scores = cbg_scores[cbg_scores.index.isin(philadelphia_cbgs)]

In [ ]:
# We'll be using google maps geocode service to get the address
# for each lat long as required by walk score api
MAPS_API='https://maps.googleapis.com/maps/api/geocode/json'
MAPS_API_KEY = ''

In [ ]:
query = {
    'key': MAPS_API_KEY
}

def get_address(record):
    query['latlng'] = '{},{}'.format(record.latitude,record.longitude)
    try:
        resp = requests.get(MAPS_API, params = query).json()
        return resp['results'][0]['formatted_address'][:-5].replace(',','')
    except:
        return 0 # these missing cbgs will be tossed

In [ ]:
cbg_scores['address'] = cbg_scores.apply(get_address, axis=1)

In [ ]:
WS_URL = 'http://api.walkscore.com'
WS_API_KEY=''

In [ ]:
def get_ws(record):
    try:
        api_url = ("{}/score?format=json&address={}&lat={}&lon={}&transit=1&wsapikey={}").format(
            WS_URL, record.address.replace(' ', '%20'), record.latitude, record.longitude, WS_API_KEY)
        response = urllib.request.urlopen(api_url)
        resp = json.loads(response.read().decode('utf-8'))
        return '{},{}'.format(resp['walkscore'], resp['transit']['score'])
    except:
        return 'NA' # these missing cbgs will be tossed

In [ ]:
cbg_scores['scores'] = cbg_scores.apply(get_ws,axis=1)
cbg_scores[['walk_score', 'transit_score']] = cbg_scores['scores'].str.split(',', expand=True)

In [166]:
del(cbg_scores['latitude'])
del(cbg_scores['longitude'])
del(cbg_scores['address'])
del(cbg_scores['scores'])

In [172]:
cbg_scores.head()

,walk_score,transit_score
census_block_group,,
421010001001,100,100
421010001002,97,100
421010002001,99,100
421010003001,96,100
421010003002,94,100


## Limited Service Restaurant Location Data
For our predictor data, we have purchased data on limited service restaurants in Philadelphia that we will use as the total of fast food locations in each Philadelphia CBG, and also to gain counts of total visitors to fastfood locations within the cbg. This notebook was run with this data located next to this notebook in a folder named 'restaurant_pattern_data' but cannot be included in version control due to proprietary license.

As the safegraph data contains an address for each location, but not the corresponding CBG, we will be using the US Census Geocoding Service to obtain the CBG for each address.

In [308]:
# First obtain the addresses and total visitor count for each fast food locations
# Column in Patterns File - Column Header
# 2 - Address
# 9 - Total visitor count
fast_food_locations = pd.read_csv('restaurant_pattern_data/patterns.csv', usecols=[2,9])
fast_food_locations.head()

,street_address,raw_visit_counts
0,399 Franklin Mills Cir,29
1,2422 W Passyunk Ave,215
2,1755 N 13th St # 1785,864
3,24 S 18th St,286
4,263 S 17th St,172


In [309]:
# next we need to use US census geocoding to obtain the CBGs for each
# of these locations
GEOCODE_API = 'https://geocoding.geo.census.gov/geocoder/geographies/address'
query = {
    'city': 'Philadelphia',
    'state': 'PA',
    'benchmark':'Public_AR_Census2010',
    'vintage':'Census2010_Census2010',
    'layers':'14',
    'format':'json'
}

def get_cbg(address):
    query['street'] = address
    resp = requests.get(GEOCODE_API, params = query).json()
    try:
        return resp['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['GEOID'][:-3]
    except (IndexError, KeyError) as e:
        return 0 # these missing cbgs will be tossed


fast_food_locations['cbg'] =\
    fast_food_locations['street_address'].apply(get_cbg)
fast_food_locations.head()

,street_address,raw_visit_counts,cbg
0,399 Franklin Mills Cir,29,421010363011
1,2422 W Passyunk Ave,215,421010038002
2,1755 N 13th St # 1785,864,421010029001
3,24 S 18th St,286,421010004022
4,263 S 17th St,172,421010008041


In [315]:
# from this we can obtain our amount of restaurants in each cbgx
# note we'll be dropping a few restaurants that failed in geocode
# lookup
cbg_fast_food_counts = fast_food_locations\
    .groupby('cbg')\
    .agg({'street_address': 'count', 'raw_visit_counts': 'sum'})
# below reindex removes entries without cbgs
cbg_fast_food_counts = cbg_fast_food_counts.reindex(philadelphia_cbgs)
cbg_fast_food_counts.index = cbg_fast_food_counts.index.map(int)
cbg_fast_food_counts = cbg_fast_food_counts.sort_index()
cbg_fast_food_counts = cbg_fast_food_counts.fillna(0)
cbg_fast_food_counts.index.name = 'census_block_group'
cbg_fast_food_counts.columns = ['Fast Food Count', 'Fast Food Consumers']
cbg_fast_food_counts.head()

,Fast Food Count,Fast Food Consumers
census_block_group,,
421010001001,1.0,1121.0
421010001002,1.0,710.0
421010002001,3.0,1370.0
421010003001,3.0,2106.0
421010003002,0.0,0.0


## Neighborhood Patterns
Included in our safegraph census data is a October 2018 summary of neighborhood patterns. We want to use this file to obtain one more variable, the total monthly visitor count to cbgs.

In [160]:
# Capture Column - Column Name
# 0 - CBG
# 3 - Total Visitor Count
neighborhood_patterns = pd.read_csv(
    'safegraph_open_census_data/data/cbg_patterns.csv',
    index_col=0, usecols=[0,3])
neighborhood_patterns =\
    neighborhood_patterns[neighborhood_patterns.index.isin(philadelphia_cbgs)]
neighborhood_patterns = neighborhood_patterns.sort_index()
neighborhood_patterns.index = neighborhood_patterns.index.map(int)
neighborhood_patterns.head()

,raw_visit_count
census_block_group,
421010001001,138223.0
421010001002,253908.0
421010002001,227669.0
421010003001,283647.0
421010003002,102105.0


In [176]:
print(len(total_pop_df))
print(len(pop_wfh))
print(len(living_alone))
print(len(single_parent))
print(len(married_parents))
print(len(school_enrollment))
print(len(high_school_achieved))
print(len(degree_achieved))
print(len(houses_in_poverty))
print(len(below_40k))
print(len(between_40_100k))
print(len(above_100k))
print(len(unemployed))
print(len(renter_occupied))
print(len(owner_occupied))
print(len(cbg_fast_food_counts))
print(len(neighborhood_patterns))
print(len(cbg_crime_counts))
print(len(cbg_scores))

1324
1324
1324
1324
1324
1324
1324
1324
1324
1324
1324
1324
1324
1324
1324
1324
1324
1324
1324


In [177]:
# Finally concatenate all our variables together
concatenated = pd.concat(
    [total_pop_df, pop_wfh, living_alone, single_parent, married_parents,
     school_enrollment, high_school_achieved, degree_achieved,
     houses_in_poverty, below_40k, between_40_100k, above_100k, unemployed,
     renter_occupied, owner_occupied, cbg_crime_counts, cbg_scores, 
     cbg_fast_food_counts, neighborhood_patterns],
    axis=1)
concatenated.head()

,Total Population,Pop. that Works from Home,Single Occupant Households,Single Parent Households,Married Parent Households,Undergrads and Grads,Diploma/GED attained,Degree attained,Households in Poverty,Below 40k Earners,...,Above 100k Earners,Unemployed,Renter Occupied Households,Owner Occupied Households,Crime Counts,walk_score,transit_score,Fast Food Count,Fast Food Consumers,raw_visit_count
421010001001,1762,47,634,13,45,236,74,1145,1123,257,...,587,0,853,270,334,100,100,1.0,1121.0,138223.0
421010001002,2250,59,564,0,51,202,167,1405,1182,174,...,611,82,708,474,178,97,100,1.0,710.0,253908.0
421010002001,2563,8,313,0,195,445,308,955,950,425,...,300,119,592,358,547,99,100,3.0,1370.0,227669.0
421010003001,942,45,219,0,18,260,21,442,437,117,...,238,30,428,9,587,96,100,3.0,2106.0,283647.0
421010003002,2588,102,616,26,117,294,259,1331,1328,373,...,596,72,898,430,102,94,100,0.0,0.0,102105.0


In [178]:
# Before writing this table lets check if there are nulls to deal with
concatenated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1324 entries, 421010001001 to 421010390008
Data columns (total 21 columns):
Total Population              1324 non-null int64
Pop. that Works from Home     1324 non-null int64
Single Occupant Households    1324 non-null int64
Single Parent Households      1324 non-null int64
Married Parent Households     1324 non-null int64
Undergrads and Grads          1324 non-null int64
Diploma/GED attained          1324 non-null int64
Degree attained               1324 non-null int64
Households in Poverty         1324 non-null int64
Below 40k Earners             1324 non-null int64
40k-100k Earners              1324 non-null int64
Above 100k Earners            1324 non-null int64
Unemployed                    1324 non-null int64
Renter Occupied Households    1324 non-null int64
Owner Occupied Households     1324 non-null int64
Crime Counts                  1324 non-null int64
walk_score                    1323 non-null object
transit_score          

In [8]:
# No nulls reported, but lets do a sanity check on the presense of
# values
concatenated[concatenated['Total Population'] == 0]

,Total Population,Pop. that Works from Home,Single Occupant Households,Single Parent Households,Married Parent Households,Undergrads and Grads,Diploma/GED attained,Degree attained,Households in Poverty,Below 40k Earners,40k-100k Earners,Above 100k Earners,Unemployed,Renter Occupied Households,Owner Occupied Households,Fast Food Count,Fast Food Consumers,raw_visit_count
census_block_group,,,,,,,,,,,,,,,,,,
421010050001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,82941.0
421019803001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,2059.0,224052.0
421019804001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,231482.0
421019805001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,82302.0
421019806001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2865.0,135416.0
421019807001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,1087.0,452841.0
421019808001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,68731.0


In [11]:
# We should remove these rows since all predictor variables are
# missing, but we also notice appears that the CBG has made a huge
# jump for most of these values from what we were seeing before. Lets
# confirm this.
print('There are {} rows collected in total'.format(len(concatenated)))
print('There are {} rows out of the expected CBG range'.format(len(concatenated[concatenated.index // 1000000 == 421019])))

There are 1336 rows collected in total
There are 11 rows out of the expected CBG range


In [19]:
# These do indeed appear to be outliers which given the overlap may be
# related to the missing data. For consistency of data we will remove
# these outliers even where values aren't missing
concatenated = concatenated[(concatenated['Total Population'] > 0) & (concatenated.index // 1000000 == 421010)]
print('There are {} rows remaining after filtering'.format(len(concatenated)))

There are 1324 rows remaining after filtering


In [179]:
# looks like we're good, lets write our final table
concatenated.to_csv('location_data_wrangled.csv')